In [ ]:
# 예제 : https://teddylee777.github.io/langchain/rag-tutorial/
#
# 예제: https://teddylee777.github.io/langchain/langchain-tutorial-02/
#
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}',  HUGGINGFACEHUB_API_TOKEN='{API_TOKEN}'식으로 각각 입력한다.
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25

In [ ]:
import os

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}',  HUGGINGFACEHUB_API_TOKEN='{API_TOKEN}'식으로 각각 입력한다.
# 
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [ ]:
# 단계1: 폴더내 모든 문서 로딩
#!pip install unstructured

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="./doc_test/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[1].page_content[:200]}")
print(f"\n[metadata]\n{docs[1].metadata}\n")

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [ ]:
# 단계 3: 임베딩
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
#vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.
# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings(model="text-embedding-3-small"))
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)


In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)



In [ ]:
# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "결혼일때 유급 휴가는 며칠 주어지나요?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

In [ ]:
# HuggingFaceHub 객체 생성
# 모델들은 아래 사이트 참조
# https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard
from langchain.llms import HuggingFaceHub

#repo_id = "google/flan-t5-xxl"
repo_id = "hwkwon/S-SOLAR-10.7B-v1.5"

llm_model = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 512}
)

In [ ]:
llm_model.invoke("한국의 수도는 어디인가요?")

In [1]:
# 예: https://teddylee777.github.io/langchain/langchain-tutorial-02/
#모델을 직접 다운로드 후 로컬(local)에서 추론Permalink
#이전 방식은 허깅페이스 서버에서 선택된 모델로 추론하고, 이에 대한 답변을 반환받는 방식입니다.
#추론 방식이 간편하지만, 서버의 성능에 따라 다르지만 추론 속도가 대체적으로 오래 걸리는 편입니다. 
#따라서, 결과를 받는데 시간이 오래 걸리거나, 혹은 답변의 지연시간이 긴 경우, Timeout 에러가 발생할 수 있습니다
#만약, 좋은 성능의 GPU 를 탑재한 서버가 있다면, 로컬에 모델을 직접 다운로드 받아 GPU 부스트를 받아서 추론할 수 있습니다. 아래는 예시코드 입니다.

import os
# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)
# os.environ['HF_HOME'] = '/home/jovyan/work/tmp'
os.environ['HF_HOME'] = './model'

In [ ]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

# HuggingFace Model ID
model_id = 'beomi/llama-2-ko-7b'
#gwonny/nox-solar-10.7b-v4-kolon-all-5-v2.0 


# HuggingFacePipeline 객체 생성
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id, 
    device=0,               # -1: CPU(default), 0번 부터는 CUDA 디바이스 번호 지정시 GPU 사용하여 추론
    task="text-generation", # 텍스트 생성
    model_kwargs={"temperature": 0.1, 
                  "max_length": 512},
)

# 템플릿
template = """질문: {question}

답변: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# 실행
question = "대한민국의 수도는 어디야?"
print(llm_chain.run(question=question))

#서울입니다. 계획도시로 잘 만들어진 도시입니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​